<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/clean_gfql_cpu_gpu_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GFQL CPU, GPU Benchmark

This notebook examines GFQL property graph query performance on 1-8 hop queries using CPU + GPU modes on various real-world 100K - 100M edge graphs. The data comes from a variety of popular social networks. The single-threaded CPU mode benefits from GFQL's novel dataframe engine, and the GPU mode further adds single-GPU acceleration. Both the `chain()` and `hop()` methods are examined.

The benchmark does not examine bigger-than-memory and distributed scenarios. The provided results here are from running on a free Google Colab T4 runtime, with a 2.2GHz Intel CPU (12 GB CPU RAM) and T4 Nvidia GPU (16 GB GPU RAM).

## Data
From [SNAP](https://snap.stanford.edu/data/)

| Network | Nodes     | Edges        |
|-------------|-----------|--------------|
| **Facebook**| 4,039     | 88,234       |
| **Twitter** | 81,306    | 2,420,766    |
| **GPlus**   | 107,614   | 30,494,866   |
| **Orkut**   | 3,072,441 | 117,185,082  |

## Results

Definitions:

* GTEPS: Giga (billion) edges traversed per second

* T edges / \$: Estimated trillion edges traversed for 1\$ USD based on observed GTEPS and a 3yr AWS reservation (as of 12/2023)

Tasks:

1. `chain()` - includes complex pre/post processing

  **Task**: `g.chain([n({'id': some_id}), e_forward(hops=some_n)])`


| **Dataset** | Max GPU Speedup      | CPU GTEPS   | GPU GTEPS   | T CPU edges / \$ (t3.l) | T GPU edges / \$ (g4dn.xl) |
|-------------|--------------|-------------|-------------|----------------------------|--------------------------------|
| **Facebook**| 1.1X  | 0.66 | 0.61 | 65.7                | 10.4                    |
| **Twitter** | 17.4X   | 0.17 | 2.81 | 16.7                | 48.1                    |
| **GPlus**   | 43.8X  | 0.09 | 2.87 | 8.5                | 49.2                    |
| **Orkut**   | N/A            | N/A         | 12.15 | N/A                        | 208.3                    |
| **AVG** | 20.7X | 0.30 | 4.61 | 30.3 | 79.0
| **MAX** | 43.8X | 0.66 | 12.15 | 65.7 | 208.3


2. `hop()` - core property search primitive similar to BFS

  **Task**: `g.hop(nodes=[some_id], direction='forward', hops=some_n)`


| **Dataset** | Max GPU Speedup | CPU GTEPS | GPU GTEPS | T CPU edges / \$ (t3.l) | T GPU edges / \$ (g4dn.xl) |
|-------------|-------------|-----------|-----------|--------------------|--------------------------------|
| **Facebook**| 3X          | 0.47      | 1.47     | 47.0        | 25.2                    |
| **Twitter** | 42X         | 0.50      | 10.51      | 50.2        | 180.2                    |
| **GPlus**   | 21X         | 0.26      | 4.11       | 26.2        | 70.4                    |
| **Orkut**   | N/A         | N/A       | 41.50     | N/A                | 711.4                    |
| **AVG** | 22X | 0.41 | 14.4 | 41.1 | 246.8
| **MAX** | 42X | 0.50 | 41.50 | 50.2 | 711.4


## Optional: GPU setup - Google Colab

In [ ]:
# Report GPU used when GPU benchmarking
! nvidia-smi

Mon Feb 19 04:14:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# if in google colab
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12 cudf-cu12 #==23.12.00 #cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11


In [ ]:
import cudf
cudf.__version__

'24.02.01'

# 1. Install & configure

In [ ]:
#! pip install graphistry[igraph]

!pip install -q igraph
!pip install -q graphistry


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 6.9 MB/s eta 0:00:00


## Imports

In [ ]:
import pandas as pd

import graphistry

from graphistry import (

    # graph operators
    n, e_undirected, e_forward, e_reverse,

    # attribute predicates
    is_in, ge, startswith, contains, match as match_re
)
graphistry.__version__

'0.33.0'

In [ ]:
import cudf

In [ ]:
#work around google colab shell encoding bugs

import locale
locale.getpreferredencoding = lambda: "UTF-8"

# 2. Perf benchmarks

### Facebook: 88K edges

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/graphistry/pygraphistry/master/demos/data/facebook_combined.txt', sep=' ', names=['s', 'd'])
print(df.shape)
df.head(5)

(88234, 2)


,s,d
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [ ]:
fg = graphistry.edges(df, 's', 'd').materialize_nodes()
print(fg._nodes.shape, fg._edges.shape)
fg._nodes.head(5)

(4039, 1) (88234, 2)


,id
0,0
1,1
2,2
3,3
4,4


already with simple 2 hop `chain` comparison we see a slight/negligable speedup enabled by setting g. to `cudf`

In [ ]:
%%time
for i in range(100):
  fg2 = fg.chain([n({'id': 0}), e_forward(hops=2)])

CPU times: user 11.2 s, sys: 1.63 s, total: 12.8 s
Wall time: 15.6 s


In [ ]:
%%time
fg_gdf = fg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(100):
  fg2 = fg_gdf.chain([n({'id': 0}), e_forward(hops=2)])
print(fg._nodes.shape, fg._edges.shape)
print(fg2._nodes.shape, fg2._edges.shape)
del fg_gdf
del fg2

(4039, 1) (88234, 2)
(1519, 1) (4060, 2)
CPU times: user 11 s, sys: 290 ms, total: 11.3 s
Wall time: 11.6 s


and 5 hop `chain` with fewer neighbors, over 2x

In [ ]:
%%time
for i in range(50):
  fg2 = fg.chain([n({'id': 0}), e_forward(hops=5)])
print(fg._nodes.shape, fg._edges.shape)
print(fg2._nodes.shape, fg2._edges.shape)

(4039, 1) (88234, 2)
(3829, 1) (86074, 2)
CPU times: user 16.4 s, sys: 728 ms, total: 17.1 s
Wall time: 19.5 s


In [ ]:
%%time
fg_gdf = fg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(50):
  fg2 = fg_gdf.chain([n({'id': 0}), e_forward(hops=5)])
print(fg._nodes.shape, fg._edges.shape)
print(fg2._nodes.shape, fg2._edges.shape)
del fg_gdf
del fg2

(4039, 1) (88234, 2)
(3829, 1) (86074, 2)
CPU times: user 8.46 s, sys: 131 ms, total: 8.59 s
Wall time: 8.57 s


with the original 100 neighbors but 5 hop `chain` now we are again about even with CPU

In [ ]:
%%time
for i in range(100):
  fg2 = fg.chain([e_forward(source_node_match={'id': 0}, hops=5)])

CPU times: user 11.7 s, sys: 1.23 s, total: 12.9 s
Wall time: 13.7 s


In [ ]:
%%time
fg_gdf = fg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(100):
  fg2 = fg_gdf.chain([e_forward(source_node_match={'id': 0}, hops=5)])
print(fg._nodes.shape, fg._edges.shape)
print(fg2._nodes.shape, fg2._edges.shape)
del fg_gdf
del fg2



(4039, 1) (88234, 2)
(348, 1) (347, 2)
CPU times: user 13.9 s, sys: 94.7 ms, total: 14 s
Wall time: 14.5 s


however, restricting to 2 forward hops only, we again see 2x

In [ ]:
%%time
start_nodes = pd.DataFrame({fg._node: [0]})
for i in range(100):
  fg2 = fg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(fg2._nodes.shape, fg2._edges.shape)

(1519, 1) (4060, 2)
CPU times: user 3.88 s, sys: 1.13 s, total: 5 s
Wall time: 5.08 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({fg._node: [0]})
fg_gdf = fg.nodes(cudf.from_pandas(fg._nodes)).edges(cudf.from_pandas(fg._edges))
for i in range(100):
  fg2 = fg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(fg2._nodes.shape, fg2._edges.shape)
del start_nodes
del fg_gdf
del fg2

(1519, 1) (4060, 2)
CPU times: user 2.49 s, sys: 7.04 ms, total: 2.5 s
Wall time: 2.49 s


and now on 5 hops we see 2x when only hopping forward

In [ ]:
%%time
start_nodes = pd.DataFrame({fg._node: [0]})
for i in range(100):
  fg2 = fg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
print(fg2._nodes.shape, fg2._edges.shape)

(3829, 1) (86074, 2)
CPU times: user 10.7 s, sys: 1.24 s, total: 11.9 s
Wall time: 12.7 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({fg._node: [0]})
fg_gdf = fg.nodes(cudf.from_pandas(fg._nodes)).edges(cudf.from_pandas(fg._edges))
for i in range(100):
  fg2 = fg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
print(fg2._nodes.shape, fg2._edges.shape)
del start_nodes
del fg_gdf
del fg2

(3829, 1) (86074, 2)
CPU times: user 5.65 s, sys: 212 ms, total: 5.87 s
Wall time: 5.85 s


## Twitter

- edges: 2420766
- nodes: 81306

In [ ]:
! wget 'https://snap.stanford.edu/data/twitter_combined.txt.gz'
#! curl -L 'https://snap.stanford.edu/data/twitter_combined.txt.gz' -o twitter_combined.txt.gz

--2024-02-19 05:04:53--  https://snap.stanford.edu/data/twitter_combined.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10621918 (10M) [application/x-gzip]
Saving to: ‘twitter_combined.txt.gz’

twitter_combined.tx 100%[===================>]  10.13M  22.5MB/s    in 0.5s    

2024-02-19 05:04:53 (22.5 MB/s) - ‘twitter_combined.txt.gz’ saved [10621918/10621918]



In [ ]:
! gunzip twitter_combined.txt.gz

gzip: twitter_combined.txt already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
! head -n 5 twitter_combined.txt

214328887 34428380
17116707 28465635
380580781 18996905
221036078 153460275
107830991 17868918


In [ ]:
%%time
te_df = pd.read_csv('twitter_combined.txt', sep=' ', names=['s', 'd'])
te_df.shape

CPU times: user 430 ms, sys: 53 ms, total: 483 ms
Wall time: 480 ms


(2420766, 2)

In [ ]:
import graphistry

In [ ]:
%%time
g = graphistry.edges(te_df, 's', 'd').materialize_nodes()
g._nodes.shape

CPU times: user 105 ms, sys: 29 ms, total: 134 ms
Wall time: 133 ms


(81306, 1)

Wow look at this, single hop `chain` 15x speedup!

In [ ]:
%%time
for i in range(10):
  g2 = g.chain([n({'id': 17116707}), e_forward(hops=1)])
g2._nodes.shape, g2._edges.shape

CPU times: user 8.68 s, sys: 6.41 s, total: 15.1 s
Wall time: 15.1 s


((140, 1), (615, 2))

In [ ]:
%%time
g_gdf = g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(10):
  out = g_gdf.chain([n({'id': 17116707}), e_forward(hops=1)])._nodes
print(out.shape)
del g_gdf
del out

(140, 1)
CPU times: user 814 ms, sys: 10.7 ms, total: 825 ms
Wall time: 820 ms


and with two hop `chain`

In [ ]:
%%time
for i in range(10):
  out = g.chain([n({'id': 17116707}), e_forward(hops=2)])
print(out._nodes.shape, out._edges.shape)

(2345, 1) (68536, 2)
CPU times: user 11.4 s, sys: 7.63 s, total: 19 s
Wall time: 19.8 s


In [ ]:
%%time
g_gdf = g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(10):
  out = g_gdf.chain([n({'id': 17116707}), e_forward(hops=2)])._nodes
print(out.shape)
del g_gdf
del out

(2345, 1)
CPU times: user 1.25 s, sys: 47.6 ms, total: 1.29 s
Wall time: 1.29 s


and 8 hop `chain`... we see and 20x speedup!!

In [ ]:
%%time
for i in range(10):
  out = g.chain([n({'id': 17116707}), e_forward(hops=8)])
print(out._nodes.shape, out._edges.shape)

(81304, 1) (2417796, 2)
CPU times: user 1min 41s, sys: 25.5 s, total: 2min 7s
Wall time: 2min 10s


In [ ]:
%%time
g_gdf = g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
for i in range(10):
  out = g_gdf.chain([n({'id': 17116707}), e_forward(hops=8)])._nodes
print(out.shape)
del g_gdf
del out

(81304, 1)
CPU times: user 4.8 s, sys: 1.31 s, total: 6.11 s
Wall time: 6.12 s


When we speedtest 1 hop from a specific node, we see a 5x improvement



In [ ]:
%%time
start_nodes = pd.DataFrame({g._node: [17116707]})
for i in range(10):
  g2 = g.hop(
      nodes=start_nodes,
      direction='forward',
      hops=1)
print(g2._nodes.shape, g2._edges.shape)

(140, 1) (615, 2)
CPU times: user 5.81 s, sys: 9.63 ms, total: 5.82 s
Wall time: 5.86 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({g._node: [17116707]})
g_gdf = g.nodes(cudf.from_pandas(g._nodes)).edges(cudf.from_pandas(g._edges))
for i in range(10):
  g2 = g_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del g_gdf
del g2

(61827, 1) (1473599, 2)
CPU times: user 922 ms, sys: 198 ms, total: 1.12 s
Wall time: 1.12 s


and with 2 hops the improvement grows

In [ ]:
%%time
start_nodes = pd.DataFrame({g._node: [17116707]})
for i in range(10):
  g2 = g.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(g2._nodes.shape, g2._edges.shape)

(2345, 1) (68536, 2)
CPU times: user 7.63 s, sys: 12.2 ms, total: 7.64 s
Wall time: 7.66 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({g._node: [17116707]})
g_gdf = g.nodes(cudf.from_pandas(g._nodes)).edges(cudf.from_pandas(g._edges))
for i in range(10):
  g2 = g_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del g_gdf
del g2

(2345, 1) (68536, 2)
CPU times: user 360 ms, sys: 4.94 ms, total: 365 ms
Wall time: 366 ms


and again with 8 hops starting from a specific node the speedup is around 20x!

In [ ]:
%%time
start_nodes = pd.DataFrame({g._node: [17116707]})
for i in range(10):
  g2 = g.hop(
      nodes=start_nodes,
      direction='forward',
      hops=8)
print(g2._nodes.shape, g2._edges.shape)

(81304, 1) (2417796, 2)
CPU times: user 38.4 s, sys: 3.68 s, total: 42 s
Wall time: 46.5 s


In [ ]:
`

(81304, 1) (2417796, 2)
CPU times: user 1.86 s, sys: 512 ms, total: 2.37 s
Wall time: 2.39 s


## GPlus

- edges: 30494866
- nodes: 107614

In [ ]:
! wget https://snap.stanford.edu/data/gplus_combined.txt.gz

--2024-02-19 05:15:15--  https://snap.stanford.edu/data/gplus_combined.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398930514 (380M) [application/x-gzip]
Saving to: ‘gplus_combined.txt.gz.1’

gplus_combined.txt. 100%[===================>] 380.45M  42.0MB/s    in 9.1s    

2024-02-19 05:15:25 (41.9 MB/s) - ‘gplus_combined.txt.gz.1’ saved [398930514/398930514]



In [ ]:
! gunzip gplus_combined.txt.gz

In [ ]:
%%time
ge_df = pd.read_csv('gplus_combined.txt', sep=' ', names=['s', 'd'])
print(ge_df.shape)
ge_df.head(5)

(30494866, 2)
CPU times: user 14.4 s, sys: 1.41 s, total: 15.8 s
Wall time: 15.9 s


,s,d
0,116374117927631468606,101765416973555767821
1,112188647432305746617,107727150903234299458
2,116719211656774388392,100432456209427807893
3,117421021456205115327,101096322838605097368
4,116407635616074189669,113556266482860931616


In [ ]:
%%time
gg = graphistry.edges(ge_df, 's', 'd').materialize_nodes()
gg = graphistry.edges(ge_df, 's', 'd').nodes(gg._nodes, 'id')
print(gg._edges.shape, gg._nodes.shape)
gg._nodes.head(5)

(30494866, 2) (107614, 1)
CPU times: user 7.89 s, sys: 3.24 s, total: 11.1 s
Wall time: 11.5 s


,id
0,116374117927631468606
1,112188647432305746617
2,116719211656774388392
3,117421021456205115327
4,116407635616074189669


In [ ]:
%%time
gg.chain([ n({'id': '116374117927631468606'})])._nodes

CPU times: user 434 ms, sys: 313 ms, total: 747 ms
Wall time: 746 ms


,id
0,116374117927631468606


1 hop `chain` 5x speedup

In [ ]:
%%time
out = gg.chain([ n({'id': '116374117927631468606'}), e_forward(hops=1)])._nodes
out.shape

CPU times: user 27.8 s, sys: 19 s, total: 46.8 s
Wall time: 51.7 s


(1473, 1)

In [ ]:
%%time
gg_gdf = gg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = gg_gdf.chain([ n({'id': '116374117927631468606'}), e_forward(hops=1)])
print(out._nodes.shape, out._edges.shape)
del gg_gdf
del out

(1473, 1) (13375, 2)
CPU times: user 6.31 s, sys: 2.79 s, total: 9.1 s
Wall time: 9.3 s


and 2 hop `chain` again we approach 15-20x speedups

In [ ]:
%%time
out = gg.chain([ n({'id': '116374117927631468606'}), e_forward(hops=2)])._nodes
out.shape

CPU times: user 51.4 s, sys: 17.1 s, total: 1min 8s
Wall time: 1min 22s


(44073, 1)

In [ ]:
%%time
gg_gdf = gg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = gg_gdf.chain([ n({'id': '116374117927631468606'}), e_forward(hops=2)])
print(out._nodes.shape, out._edges.shape)
del gg_gdf
del out

(44073, 1) (2069325, 2)
CPU times: user 6.15 s, sys: 2.69 s, total: 8.84 s
Wall time: 8.85 s


with 3 hop `chain` on such a large network, travesing via CPU becoming time prohibitive, not to mention a memory hog.

here we see a 30x speedup, which keeps the investigation within the realm of interactivity!

In [ ]:
%%time
out = gg.chain([ n({'id': '116374117927631468606'}), e_forward(hops=3)])._nodes
out.shape

CPU times: user 3min 48s, sys: 1min 14s, total: 5min 2s
Wall time: 5min 18s


(102414, 1)

In [ ]:
%%time
gg_gdf = gg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = gg_gdf.chain([ n({'id': '116374117927631468606'}), e_forward(hops=3)])
print(out._nodes.shape, out._edges.shape)
del gg_gdf
del out

(102414, 1) (24851333, 2)
CPU times: user 7.61 s, sys: 2.9 s, total: 10.5 s
Wall time: 10.6 s


4 hop `chain` we see nearly 40x

In [ ]:
%%time
out = gg.chain([ n({'id': '116374117927631468606'}), e_forward(hops=4)])
print(out._nodes.shape, out._edges.shape)

(105479, 1) (30450354, 2)
CPU times: user 5min 32s, sys: 1min 30s, total: 7min 2s
Wall time: 7min 19s


In [ ]:
%%time
gg_gdf = gg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = gg_gdf.chain([ n({'id': '116374117927631468606'}), e_forward(hops=4)])
print(out._nodes.shape, out._edges.shape)
del gg_gdf
del out

(105479, 1) (30450354, 2)
CPU times: user 8.12 s, sys: 2.76 s, total: 10.9 s
Wall time: 10.9 s


and 5 hop `chain` we see over 30x

In [ ]:
%%time
out = gg.chain([ n({'id': '116374117927631468606'}), e_forward(hops=5)])
print(out._nodes.shape, out._edges.shape)

(105604, 1) (30468335, 2)
CPU times: user 6min 49s, sys: 1min 45s, total: 8min 34s
Wall time: 8min 36s


In [ ]:
%%time
gg_gdf = gg.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = gg_gdf.chain([ n({'id': '116374117927631468606'}), e_forward(hops=5)])
print(out._nodes.shape, out._edges.shape)
del gg_gdf
del out

(105604, 1) (30468335, 2)
CPU times: user 11 s, sys: 3.42 s, total: 14.4 s
Wall time: 14.5 s


1 hop forward yields 2-3x

In [ ]:
%%time
start_nodes = pd.DataFrame({gg._node: ['116374117927631468606']})
for i in range(1):
  g2 = gg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=1)
print(g2._nodes.shape, g2._edges.shape)

(1473, 1) (13375, 2)
CPU times: user 17 s, sys: 7.27 s, total: 24.2 s
Wall time: 24.4 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({gg._node: ['116374117927631468606']})
gg_gdf = gg.nodes(cudf.from_pandas(gg._nodes)).edges(cudf.from_pandas(gg._edges))
for i in range(1):
  g2 = gg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=1)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del gg_gdf
del g2

(1473, 1) (13375, 2)
CPU times: user 6.56 s, sys: 2.94 s, total: 9.5 s
Wall time: 9.61 s


2 hops forward 5x

In [ ]:
%%time
start_nodes = pd.DataFrame({gg._node: ['116374117927631468606']})
for i in range(1):
  g2 = gg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(g2._nodes.shape, g2._edges.shape)

(44073, 1) (2069325, 2)
CPU times: user 26 s, sys: 11.3 s, total: 37.3 s
Wall time: 37.4 s


In [ ]:
%%time
start_nodes = cudf.DataFrame({gg._node: ['116374117927631468606']})
gg_gdf = gg.nodes(cudf.from_pandas(gg._nodes)).edges(cudf.from_pandas(gg._edges))
for i in range(1):
  g2 = gg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del gg_gdf
del g2

(44073, 1) (2069325, 2)
CPU times: user 4.62 s, sys: 2.3 s, total: 6.92 s
Wall time: 6.9 s


3 hops forward 8x

In [ ]:
%%time
start_nodes = pd.DataFrame({gg._node: ['116374117927631468606']})
for i in range(1):
  g2 = gg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=3)
print(g2._nodes.shape, g2._edges.shape)

(102414, 1) (24851333, 2)
CPU times: user 1min 5s, sys: 19.2 s, total: 1min 25s
Wall time: 1min 25s


In [ ]:
%%time
start_nodes = cudf.DataFrame({gg._node: ['116374117927631468606']})
gg_gdf = gg.nodes(cudf.from_pandas(gg._nodes)).edges(cudf.from_pandas(gg._edges))
for i in range(1):
  g2 = gg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=3)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del gg_gdf
del g2

(102414, 1) (24851333, 2)
CPU times: user 7.44 s, sys: 3.06 s, total: 10.5 s
Wall time: 10.6 s


4 hops forward nearly 15x

In [ ]:
%%time
start_nodes = pd.DataFrame({gg._node: ['116374117927631468606']})
for i in range(1):
  g2 = gg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=4)
print(g2._nodes.shape, g2._edges.shape)

(105479, 1) (30450354, 2)
CPU times: user 1min 46s, sys: 27.2 s, total: 2min 13s
Wall time: 2min 14s


In [ ]:
%%time
start_nodes = cudf.DataFrame({gg._node: ['116374117927631468606']})
gg_gdf = gg.nodes(cudf.from_pandas(gg._nodes)).edges(cudf.from_pandas(gg._edges))
for i in range(1):
  g2 = gg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=4)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del gg_gdf
del g2

(105479, 1) (30450354, 2)
CPU times: user 6.35 s, sys: 2.62 s, total: 8.97 s
Wall time: 8.98 s


5 hops forward 15x again

In [ ]:
%%time
start_nodes = pd.DataFrame({gg._node: ['116374117927631468606']})
for i in range(1):
  g2 = gg.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
print(g2._nodes.shape, g2._edges.shape)

(105604, 1) (30468335, 2)
CPU times: user 2min 26s, sys: 36.9 s, total: 3min 3s
Wall time: 3min 18s


In [ ]:
%%time
start_nodes = cudf.DataFrame({gg._node: ['116374117927631468606']})
gg_gdf = gg.nodes(cudf.from_pandas(gg._nodes)).edges(cudf.from_pandas(gg._edges))
for i in range(1):
  g2 = gg_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del gg_gdf
del g2

(105604, 1) (30468335, 2)
CPU times: user 6.64 s, sys: 2.85 s, total: 9.48 s
Wall time: 11.8 s


## Orkut
- 117M edges
- 3M nodes

In [ ]:
! wget https://snap.stanford.edu/data/bigdata/communities/com-orkut.ungraph.txt.gz

--2024-02-19 06:02:00--  https://snap.stanford.edu/data/bigdata/communities/com-orkut.ungraph.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 447251958 (427M) [application/x-gzip]
Saving to: ‘com-orkut.ungraph.txt.gz’

com-orkut.ungraph.t 100%[===================>] 426.53M  31.8MB/s    in 11s     

2024-02-19 06:02:11 (37.4 MB/s) - ‘com-orkut.ungraph.txt.gz’ saved [447251958/447251958]



In [ ]:
! gunzip com-orkut.ungraph.txt.gz

In [ ]:
! head -n 7 com-orkut.ungraph.txt

# Undirected graph: ../../data/output/orkut.txt
# Orkut
# Nodes: 3072441 Edges: 117185083
# FromNodeId	ToNodeId
1	2
1	3
1	4


In [ ]:
import pandas as pd

import graphistry

from graphistry import (

    # graph operators
    n, e_undirected, e_forward, e_reverse,

    # attribute predicates
    is_in, ge, startswith, contains, match as match_re
)

import cudf

#work around google colab shell encoding bugs
import locale
locale.getpreferredencoding = lambda: "UTF-8"

cudf.__version__, graphistry.__version__

('24.02.01', '0.33.0')

In [ ]:
! nvidia-smi

Mon Feb 19 06:02:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0              29W /  70W |    111MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
co_df = cudf.read_csv('com-orkut.ungraph.txt', sep='\t', names=['s', 'd'], skiprows=5).to_pandas()
print(co_df.shape)
print(co_df.head(5))
print(co_df.dtypes)
#del co_df

(117185082, 2)
   s  d
0  1  3
1  1  4
2  1  5
3  1  6
4  1  7
s    int64
d    int64
dtype: object
CPU times: user 2.34 s, sys: 1.29 s, total: 3.63 s
Wall time: 3.77 s


from load into gpu and back to cpu again

In [ ]:
%%time
co_g = graphistry.edges(cudf.DataFrame(co_df), 's', 'd').materialize_nodes(engine='cudf')
co_g = co_g.nodes(lambda g: g._nodes.to_pandas()).edges(lambda g: g._edges.to_pandas())
print(co_g._nodes.shape, co_g._edges.shape)
co_g._nodes.head(5)

(3072441, 1) (117185082, 2)
CPU times: user 2.06 s, sys: 7.93 s, total: 10 s
Wall time: 11.2 s


,id
0,1
1,2
2,3
3,4
4,5


In [ ]:
! nvidia-smi

Mon Feb 19 06:03:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              29W /  70W |   2825MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

1 hop

In [ ]:
%%time
# crashes
if False:
  out = co_g.chain([ n({'id': 1}), e_forward(hops=1)])._nodes
  print(out.shape)
  del out

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  out = co_gdf.chain([ n({'id': 1}), e_forward(hops=1)])
! nvidia-smi
print(out._nodes.shape, out._edges.shape)
del co_gdf
del out

Mon Feb 19 06:03:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              29W /  70W |   1929MiB / 15360MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  out = co_gdf.chain([ n({'id': 1}), e_forward(hops=2)])
! nvidia-smi
print(out._nodes.shape, out._edges.shape)
del co_gdf
del out

Mon Feb 19 06:03:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0              39W /  70W |   1929MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

1 hop

In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  out = co_gdf.chain([ n({'id': 1}), e_forward(hops=3)])
! nvidia-smi
print(out._nodes.shape, out._edges.shape)
del co_gdf
del out

Mon Feb 19 06:04:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0              29W /  70W |   1929MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  out = co_gdf.chain([ n({'id': 1}), e_forward(hops=4)])
! nvidia-smi
print(out._nodes.shape, out._edges.shape)
del co_gdf
del out

Mon Feb 19 06:04:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0              40W /  70W |   1929MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  out = co_gdf.chain([ n({'id': 1}), e_forward(hops=5)])
! nvidia-smi
print(out._nodes.shape, out._edges.shape)
del co_gdf
del out

Mon Feb 19 06:05:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0              43W /  70W |   1929MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
out = co_gdf.chain([ n({'id': 1}), e_forward(hops=6)])._nodes
print(out.shape)
del co_gdf
del out

In [ ]:
!lscpu


Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [ ]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       5.8Gi       1.6Gi       1.0Gi       5.2Gi       5.5Gi
Swap:             0B          0B          0B


GPU memory begins to be an issue, as demonstrated when one does not explicitly delete large cudf objects

In [ ]:
%%time
start_nodes = pd.DataFrame({'id': [1]})
! nvidia-smi
for i in range(1):
  g2 = co_g.hop(
      nodes=start_nodes,
      direction='forward',
      hops=1)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
#del start_nodes
#del co_gdf
#del g2

Mon Feb 19 06:05:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              43W /  70W |  13663MiB / 15360MiB |     13%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=1)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:17:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0              28W /  70W |   1925MiB / 15360MiB |     36%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=2)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:17:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0              27W /  70W |   1925MiB / 15360MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=3)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:18:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              28W /  70W |   1927MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=4)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:18:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              28W /  70W |   1927MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=5)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:18:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              36W /  70W |   1927MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
%%time
start_nodes = cudf.DataFrame({'id': [1]})
co_gdf = co_g.nodes(lambda g: cudf.DataFrame(g._nodes)).edges(lambda g: cudf.DataFrame(g._edges))
! nvidia-smi
for i in range(10):
  g2 = co_gdf.hop(
      nodes=start_nodes,
      direction='forward',
      hops=6)
! nvidia-smi
print(g2._nodes.shape, g2._edges.shape)
del start_nodes
del co_gdf
del g2

Mon Feb 19 06:18:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0              37W /  70W |   1927MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--